In [1]:
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


In [2]:
train = pd.read_csv('D:/Earthquake Prediction/data/train.csv', nrows=300000000,
                  dtype={'acoustic_data':np.int32,'time_to_failure':np.float64})

In [3]:
train_data = train.values
train.head()

,acoustic_data,time_to_failure
0,12,1.4691
1,6,1.4691
2,8,1.4691
3,5,1.4691
4,8,1.4691


In [4]:
test_seg=pd.read_csv('D:/Earthquake Prediction/data/test/seg_0a0fbb.csv',dtype={'acoustic_data':np.int32})
test_seg.head()
test_seg.shape


(150000, 1)

In [5]:
# data greet. sample data by step 50.
train_acoustic = train['acoustic_data'].values
train_failure = train['time_to_failure'].values

fig,ax = plt.subplots(2, 1, figsize=(16, 10))
ax[0].plot(train_acoustic, color='b')
ax[0].set_title('acoustic_data')
ax[0].set_xlabel('index')

ax[1].plot(train_failure, color='b')
ax[1].set_title('time_to_failure')
ax[1].set_xlabel('index')


Text(0.5,0,'index')

In [5]:
def extract_features(x):
    return np.c_[x.mean(axis=1), x.std(axis=1), x.min(axis=1), x.max(axis=1)]

In [6]:
def one_sample_creation(x, last_index=None, n_step=150, seq_length=1000):
    if last_index is None:
        last_index = len(x)
    assert last_index - n_step*seq_length >= 0
    Temp = (x[last_index -n_step*seq_length:last_index].reshape(n_step, -1) - 5)/10
    
    return extract_features(Temp)

n_features = one_sample_creation(train_data).shape[1]

In [7]:
def train_data_generator(data, n_step=150, seq_length=1000):
    len_data = len(data)
    sample_length = n_step*seq_length
    rows = [x for x in range(sample_length, len_data+1)[::sample_length]]
    samples = np.zeros((len(rows), n_step, n_features))
    targets = np.zeros((len(rows),))
    for i, row in enumerate(rows):
        samples[i] = one_sample_creation(data[:,0], last_index=row, n_step=n_step, seq_length=seq_length)
        targets[i] = data[row-1,1]
    return samples, targets

In [9]:
n_step = 150
seq_length = 1000
X, Y = train_data_generator(train_data, n_step=n_step, seq_length=seq_length)

second_earthquake = 50085877
validation_X, validation_Y = train_data_generator(train_data[:second_earthquake], n_step=n_step, seq_length=seq_length)

In [12]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mae')
es = EarlyStopping(monitor='val_loss', mode='min', patience=5)
model.fit(X, Y, validation_data=(validation_X, validation_Y), batch_size=32, epochs=50, callbacks=[es])

Train on 2000 samples, validate on 333 samples

Epoch 1/50

  32/2000 [..............................]

 - ETA: 1:33 - loss: 5.6400



...

1984/2000 [============================>.]

 - ETA: 0s - loss: 3.2298



2000/2000 [==============================]

 - 12s 6ms/step - loss: 3.2311 - val_loss: 3.1007


In [13]:
submission = pd.read_csv('D:/Earthquake Prediction/data/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

# Load each test data, create the feature matrix, get numeric prediction
for i, seg_id in enumerate(tqdm(submission.index)):
  #  print(i)
    seg = pd.read_csv('D:/Earthquake Prediction/data/test/' + seg_id + '.csv')
    x = seg['acoustic_data'].values
    submission.time_to_failure[i] = model.predict(np.expand_dims(one_sample_creation(x), 0))

submission.head()
submission.to_csv('submission.csv')

  0%|          | 0/2624 [00:00<?, ?it/s]

  0%|          | 1/2624 [00:00<10:40,  4.10it/s]

  0%|          | 4/2624 [00:00<08:01,  5.44it/s]

  0%|          | 8/2624 [00:00<06:00,  7.26it/s]

  0%|          | 12/2624 [00:00<04:38,  9.39it/s]

...

 99%|█████████▉| 2610/2624 [01:17<00:00, 34.65it/s]

100%|█████████▉| 2614/2624 [01:17<00:00, 34.69it/s]

100%|█████████▉| 2618/2624 [01:17<00:00, 34.81it/s]

100%|█████████▉| 2622/2624 [01:17<00:00, 34.71it/s]

100%|██████████| 2624/2624 [01:17<00:00, 33.72it/s]